# 4. Training a classifier

## What about data?

보통 이미지나 텍스트, 오디오, 그리고 비디오 데이터를 처리하기 위해서는 일단 표준 파이썬 패키지를 이용해서 numpy array 형식으로 이들을 불러와서 Tensor로 변환하는 방법을 사용한다.

`torchvision` 패키지에서는 CIFAR10, MNIST와 이미지 데이터를 변환하는 함수들을 가지고 있다. 즉 `torchvision.datasets` 패키지와 `torchvision.transforms` 패키지, 그리고 `torch.utils.data.DataLoader` 패키지를 사용한다.

이 튜토리얼에서는 CIFAR10 데이터셋을 사용하게 될 것이다. 사진 10당 label하나로 이루어진 이미지 데이터셋이다.

## Training an image classifier 

아래의 절차를 순서대로 따르며 문제를 해결해보자.

1. `torchvision` 패키지를 이용해 CIFAR10 데이터셋을 불러오고 normalizing을 한다.
2. CNN을 만든다.
3. loss function을 만든다.
4. 만든 모델을 훈련시킨다.
5. 테스트

### 1. Loading and normalizing CIFAR10 

`torchvision` 패키지를 이용해서 CIFAR10 데이터셋을 불러오자.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

torchvision 패키지로 불러온 데이터셋은 [0, 1]의 범위를 갖는 이미지인데, Tensor로 바꾸고, [-1, 1]까지의 범위로 normalize를 시킬 것이다.

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog',
           'frog', 'horse', 'ship', 'truck')

`matplotlib` 패키지를 이용해서 이미지를 확인해보자.

In [5]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5 # normalize를 다시 해제한다.
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

  dog  deer truck  frog


### 2. Define a Convolution Neural Network

In [16]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

### 3. Define a Loss function and optimizer 

In [19]:
import torch.optim as optim

cost_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

### 4. Train the network 

In [20]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        cost = cost_func(outputs, labels)
        cost.backward()
        optimizer.step()
        
        running_loss += cost.data[0]
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' 
                  % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.851
[1,  4000] loss: 1.793
[1,  6000] loss: 1.780
[1,  8000] loss: 1.743
[1, 10000] loss: 1.718
[1, 12000] loss: 1.720
[2,  2000] loss: 1.672
[2,  4000] loss: 1.647
[2,  6000] loss: 1.667
[2,  8000] loss: 1.634
[2, 10000] loss: 1.651
[2, 12000] loss: 1.600
Finished Training


### 5. Test the network on the test data 

In [21]:
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruch:', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

GroundTruch:   cat  ship  ship plane


훈련된 모델이 어떤 결론을 내리는지 확인해보자.

In [22]:
outputs = net(Variable(images))

모델이 추측한 내용을 알기 위해서는 각 class에 대해 추측한 수치 중에서 가장 높은 수치를 찾아야 한다.

In [23]:
_, predicted = torch.max(outputs.data, 1)

print('Predicted:', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

Predicted: truck  ship  ship plane


얼마나 잘 맞추는지 정확도를 구해보자

In [24]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 38 %


In [25]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 45 %
Accuracy of   car : 69 %
Accuracy of  bird : 24 %
Accuracy of   cat : 44 %
Accuracy of  deer : 32 %
Accuracy of   dog :  6 %
Accuracy of  frog : 53 %
Accuracy of horse : 50 %
Accuracy of  ship : 35 %
Accuracy of truck : 25 %


## Training on GPU 

neural net을 GPU상에서 돌리기 위해서는

를 하면 된다. 이렇게 하면 net안에 존재하는 모든 parameter들과 buffer들을 CUDA Tensor로 바꾸게 된다.

모델을 GPU상에 돌리면 이 모델에 대한 입력 또한 마찬가지로 GPU상에서 돌려야 한다. 근데 사실 네트워크의 크기가 작다면 속도 상승을 체감하지 못할 것이다.

이제 Tensor에 대한 라이브러리와 neural network에 대해서 더 공부해보자.